In [ ]:
from bs4 import BeautifulSoup
import requests
import html5lib
import sqlite3
import uuid
import re
import json
import time
import concurrent.futures

In [ ]:
connection = sqlite3.connect('MovieData.db')
cur = connection.cursor()

In [ ]:
table1 = 'CREATE TABLE IF NOT EXISTS MovieIndex(Id, Name, Url)'
cur.execute(table1)

In [ ]:
baseurl = 'https://tamilpaatu.com'
requesturl = baseurl + '/movie-index'

In [ ]:
def getsoup(url):
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    source = requests.get(url, headers= headers)
    return BeautifulSoup(source.content, 'html5lib')

In [ ]:
movieindexpage = getsoup(requesturl)
tagDiv = movieindexpage.find('div', class_='tagdiv')
movieIndex = tagDiv.find_all('a')
for el in movieIndex:
    indexname = el.text
    url = baseurl + el.get('href')
    indexquery = 'SELECT 1 FROM MovieIndex WHERE Url = ?'
    indexres = cur.execute(indexquery, [url])
    if indexres.fetchone() is None:
        cur.execute('INSERT INTO MovieIndex VALUES (?, ?, ?)', [str(uuid.uuid4()), indexname, url])

In [ ]:
connection.commit()

In [ ]:
table2 = 'CREATE TABLE IF NOT EXISTS MoviePage(Id, Name, Url, WebpSrc, JpegSrc)'
cur.execute(table2)

In [ ]:
song_pages = []
retrive_pages = []
def page_data_parse(href):
    songpage = getsoup(href)
    songscontainer = songpage.find('div', class_='botlist')
    if songscontainer is None:
        if href not in retrive_pages:
            retrive_pages.append(href)
        return
    songs = songscontainer.find_all('div', class_='a-i')
    for song in songs:
        el = song.find('a')
        song_href = baseurl + el.get('href')
        picture = el.find('picture')
        webp_src = baseurl + picture.find_all('source')[0].get('srcset')
        jpeg_src = baseurl + picture.find_all('source')[1].get('srcset')
        info = el.find('div', class_='info')
        title = info.find('h2').text
        
        song_query = 'SELECT 1 FROM MoviePage WHERE Url = ?'
        song_res = cur.execute(song_query, [song_href])
        if song_res.fetchone() is None:
            song_insert_query = 'INSERT INTO MoviePage VALUES (?, ?, ?, ?, ?)'
            cur.execute(song_insert_query, [str(uuid.uuid4()), title, song_href, webp_src, jpeg_src])
            connection.commit()
            
        if song_href not in song_pages:
            song_pages.append(song.find('a').get('href'))

    pagination_container = songscontainer.find('nav', class_='pagination')
    next_page_container = pagination_container.find('span', class_='next')
    next_page = next_page_container.find('a')
    
    if href in retrive_pages:
        retrive_pages.remove(href)

    if next_page is None:
        return
    next_page_href = next_page.get('href')
    page_data_parse(baseurl + next_page_href)

In [ ]:
movie_index_query = 'SELECT Url FROM MovieIndex'
movie_index_res = cur.execute(movie_index_query)
movieIndexHref = movie_index_res.fetchall()
for moviepage in movieIndexHref:    
    page_data_parse(moviepage[0])

In [ ]:
while 1:
    for moviepage in retrive_pages:
        page_data_parse(moviepage)
    if len(retrive_pages) == 0:
        break

In [ ]:
def get_detail(id, uri, title):
    songdetail_soap = getsoup(uri)
    article = songdetail_soap.find('article', class_='post-content')
    if article is None:
        pass
    movie_information_ele = article.find('fieldset', id='movie-handle')
    movie_information = movie_information_ele.text.strip()
    information_collection = movie_information.split('\n')
    #Casts
    starring_text = list(filter(lambda x: 'Starring' in x, information_collection))[0].strip().split(':')[1]
    casts = [cast.strip() for cast in re.split(r',|&', starring_text)]

    #Music
    music_text = list(filter(lambda x: 'Music' in x, information_collection))[0].strip().split(':')[1]
    music_directors = [director.strip() for director in re.split(r',|&', music_text)]

    #Director
    director_text = list(filter(lambda x: 'Director' in x, information_collection))[0].strip().split(':')[1]
    directors = [director.strip() for director in re.split(r',|&', director_text)]

    #Lyrics 
    lyrics_text = list(filter(lambda x: 'Lyrics' in x, information_collection))[0].strip().split(':')[1]
    lyric_writers = [writer.strip() for writer in re.split(r',|&', lyrics_text)]

    #Year
    year = list(filter(lambda x: 'Year' in x, information_collection))[0].strip().split(':')[1]

    #Language
    language = list(filter(lambda x: 'Language' in x, information_collection))[0].strip().split(':')[1]

    #like & dislike
    rating_el = article.find('div', class_='rating')
    like = rating_el.find('button', id = 'like').find('span', id = 'lcnt').text
    dislike = rating_el.find('button', id = 'dlike').find('span', id = 'dcnt').text

    #movie songs
    track_table = article.find('table', id='tlist')
    if track_table is None:
        return None
    
    track_trs = track_table.find_all('tr', attrs={'itemprop':'itemListElement'})

    track_info_tds = [track_tr.find('td') for track_tr in track_trs]
    
    track_infos = []

    for track_info_td in track_info_tds:
        link_element = track_info_td.find('a')
        if link_element is None:
            break
        track_infos.append({"Track_Name": track_info_td.find('a').text, "Track_Href": baseurl + track_info_td.find('a').get('href')})
    
    #track_infos = [{
    #    "Track_Name": track_info_td.find('a').text,
    #    "Track_Href": baseurl + track_info_td.find('a').get('href')
    #}, track_info_td if track_info_td is not None else pass track_info_td.find('td') is not None for track_info_td in track_info_tds]

    movie_detail = {
        "id": id,
        "Name": title,
        "Casts": casts,
        "Music_Director": music_directors,
        "Director": directors,
        "Lyricist": lyric_writers,
        "Year": year.strip(),
        "Language": language.strip(),
        "Likes": like,
        "Dislikes": dislike,
        "Tracks": track_infos
    }
    return movie_detail

In [ ]:
cur.execute('ALTER TABLE MoviePage ADD COLUMN IsProcessed BIT DEFAULT 0')

In [ ]:
cur.execute('ALTER TABLE MoviePage ADD COLUMN IsErrorProcessed BIT DEFAULT 0')

In [ ]:
create_movie_detail_table_query = 'CREATE TABLE IF NOT EXISTS MovieDetail(Id, Name, Casts, MusicDirectors, Directors, Lyricists, Year, Language, Likes, Dislikes, Tracks)'
cur.execute(create_movie_detail_table_query)

In [ ]:
cur.execute('ALTER TABLE MovieDetail ADD COLUMN IsProcessed BIT DEFAULT 0')

In [ ]:
start_time = time.time()
while 1:
    select_query_movie_page = 'SELECT * FROM MoviePage WHERE IsProcessed = 0 LIMIT 10'
    movie_page = cur.execute(select_query_movie_page)
    movie_page_res = movie_page.fetchall()
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        tasks = {executor.submit(get_detail, movie_res[0], movie_res[2], movie_res[1]): movie_res for movie_res in movie_page_res}
        for task in concurrent.futures.as_completed(tasks):
            try:
                currentTask = task.result()
                if currentTask is not None:
                    #print(currentTask)
                    insert_movie_detail_query = 'INSERT INTO MovieDetail VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                    movie_detail_value = [str(uuid.uuid4()), currentTask.get('Name'), json.dumps(currentTask.get('Casts')), json.dumps(currentTask.get('Music_Director')), json.dumps(currentTask.get('Director')), json.dumps(currentTask.get('Lyricist')), currentTask.get('Year'), currentTask.get('Language'), currentTask.get('Likes'), currentTask.get('Dislikes'), json.dumps(currentTask.get('Tracks')), 0]
                    cur.execute(insert_movie_detail_query, movie_detail_value)
                    connection.commit()
                    update_movie_page_query = 'UPDATE MoviePage SET IsProcessed = 1 WHERE Id = ?'
                    cur.execute(update_movie_page_query, [currentTask.get('id')])
                    connection.commit()
            except Exception as exc:
                print(tasks[task], exc)
            
        end_time = time.time()
        time_difference = (end_time - start_time) // 60
        #print(time_difference)
        if time_difference >= 1:
            print(time_difference)
            break

In [261]:
start_time = time.time()
while 1:
    select_query_movie_page = 'SELECT * FROM MoviePage WHERE IsProcessed = 0 AND IsErrorProcessed = 0 LIMIT 1'
    movie_page = cur.execute(select_query_movie_page)
    movie_page_res = movie_page.fetchall()
    if movie_page_res is None or len(movie_page_res) == 0:
        break
    try:
        for movie_page in movie_page_res:
            currentTask = get_detail(movie_page[0], movie_page[2], movie_page[1])
            if currentTask is not None:
                #print(currentTask)
                insert_movie_detail_query = 'INSERT INTO MovieDetail VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
                movie_detail_value = [str(uuid.uuid4()), currentTask.get('Name'), json.dumps(currentTask.get('Casts')), json.dumps(currentTask.get('Music_Director')), json.dumps(currentTask.get('Director')), json.dumps(currentTask.get('Lyricist')), currentTask.get('Year'), currentTask.get('Language'), currentTask.get('Likes'), currentTask.get('Dislikes'), json.dumps(currentTask.get('Tracks')), 0]
                cur.execute(insert_movie_detail_query, movie_detail_value)
                connection.commit()
                update_movie_page_query = 'UPDATE MoviePage SET IsProcessed = 1 WHERE Id = ?'
                cur.execute(update_movie_page_query, [currentTask.get('id')])
                connection.commit()
            else:
                update_movie_page_error_query = 'UPDATE MoviePage SET IsErrorProcessed = 1 WHERE Id = ?'
                cur.execute(update_movie_page_error_query, [movie_page[0]])
                connection.commit()
    except Exception as exc:
        print(exc, movie_page_res)

    if movie_page_res is None:
        break

    end_time = time.time()
    time_difference = (end_time - start_time) // 60
    #print(time_difference)
    if time_difference >= 1:
        print(time_difference)
        break
    

In [ ]:
select_movie_query = 'SELECT * FROM MoviePage'
movie_page_res = cur.execute(select_movie_query)
movie_page_res_obj = movie_page_res.fetchone()
data = get_detail(uuid.uuid4(), 'https://tamilpaatu.com/netrikann-songs-1', 'test')
print(data)